<a href="https://colab.research.google.com/github/SJonatha/Projeto-Azul/blob/main/gpt_cptec_ecmwf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wget
!pip install pandas
!pip install numpy
!pip install eccodes
!sudo apt-get install libeccodes0
!pip install cfgrib
!python -m cfgrib selfcheck
!pip install MetPy
!pip install xarray
!pip install cartopy
!pip install mygrads
!pip install colorspace

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=70775017e5a917591c3c9337a621f2a27291f477fa1483a5c3ac231f63bb6076
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 47.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libeccodes-data
The following NEW packages will be installed:
  libeccodes-data libeccodes0
0 upgraded, 2 newly installed, 0 to remove and 29 not upgraded.
Need to get 2,206 kB of archives.
After this operation, 54.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libeccodes-data all 2.24.2-1 [1,592 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libeccodes0 amd64 2.24.2-1 [614 kB]
F

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#from matplotlib.cm import get_cmap
from matplotlib import colormaps

# from matplotlib.colors import LinearSegmentedColormap

import cartopy.crs as ccrs
import cartopy.feature as ft

from cartopy.util import add_cyclic_point
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import mygrads as mg #pip install mygrads

import metpy.calc as mpcalc
from metpy.calc import lifted_index, mixed_parcel, parcel_profile
from metpy.units import units
from numpy import concatenate

from datetime import datetime, timedelta

import pandas as pd
import os
import wget
import xarray as xr

from colorspace import sequential_hcl, desaturate

from scipy.ndimage import gaussian_filter
from scipy.interpolate import interp2d, RegularGridInterpolator

In [ ]:
hoje = datetime.today()
ano = hoje.year
mes = f"{hoje.month:02}"
dia = f"{hoje.day:02}"
# dia='11'

hora_prev = '00' # '06' '12' '18'

# horas de previsao para montagem dos gráficos
# previsoes = [f"{i:03}" for i in range(0,144,3)]+[f"{i:03}" for i in range(144,241,6)]

# horas de previsao para montagem dos gráficos (7 DIAS DE PREVISÃO)
previsoes = [f"{i:03}" for i in range(0,143,3)]#+[f"{i:03}" for i in range(144,169,6)]
hora  = pd.date_range(start=f'{ano}-{mes}-{dia}', periods=48, freq='3h')

FFF = [int(i) for i in previsoes]

dia_atual = datetime.strptime(f'{ano}-{mes}-{dia}', '%Y-%m-%d')
eixo_x = [dia_atual + timedelta(hours=i) for i in FFF]
#eixo_x = [d.strftime('%d/%m/%Y %H:%M') for d in eixo_x]
# eixo_x


In [ ]:
[wget.download(f'https://data.ecmwf.int/forecasts/{ano}{mes}{dia}/{hora_prev}z/ifs/0p25/oper/{ano}{mes}{dia}000000-{x}h-oper-fc.grib2', f'{ano}{mes}{dia}000000-{x:03}h-oper-fc.grib2') for x in FFF]

# horas de previsao
arquivo = [f'{ano}{mes}{dia}000000-{x:03}h-oper-fc.grib2' for x in FFF]

In [ ]:
# Abre setores dos gribs
grib = xr.open_mfdataset(arquivo , engine='cfgrib', concat_dim='time',combine='nested',backend_kwargs={'filter_by_keys': {'stepType': 'instant','typeOfLevel': 'isobaricInhPa'}})
grib = grib.assign_coords(time=hora) # Corrige as informações de datetime

grib1 = xr.open_mfdataset(arquivo, engine='cfgrib', concat_dim='time',combine='nested', backend_kwargs={'filter_by_keys': {'typeOfLevel': 'meanSea'}})
grib1 = grib1.assign_coords(time=hora) # Corrige as informações de datetime

grib2 = xr.open_mfdataset(arquivo, engine='cfgrib', concat_dim='time',combine='nested',filter_by_keys={'stepType':'instant' ,'typeOfLevel': 'surface'})
grib2 = grib2.assign_coords(time=hora) # Corrige as informações de datetime

grib3 = xr.open_mfdataset(arquivo , engine='cfgrib', concat_dim='time',combine='nested', filter_by_keys={'stepType': 'instant','typeOfLevel': 'entireAtmosphere'})
grib3 = grib3.assign_coords(time=hora) # Corrige as informações de datetime

grib4 = xr.open_mfdataset(arquivo , engine='cfgrib', concat_dim='time',combine='nested', filter_by_keys={'stepType': 'instant','typeOfLevel': 'mostUnstableParcel'})
grib4 = grib4.assign_coords(time=hora) # Corrige as informações de datetime

/usr/local/lib/python3.11/dist-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
/usr/local/lib/python3.11/dist-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
/usr/local/lib/python3.11/dist-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
/usr

In [ ]:
# xr.open_mfdataset(arquivo , engine='cfgrib', concat_dim='time',combine='nested').variables

# grib4 = xr.open_mfdataset(arquivo , engine='cfgrib', concat_dim='time',combine='nested', filter_by_keys={'stepType': 'instant','typeOfLevel': 'mostUnstableParcel'})
# grib4 = grib4.assign_coords(time=hora) # Corrige as informações de datetime

In [ ]:
class Calc:

    def __init__(self):
        self.size      = 10
        self.step      = 25
        self.linewidth = 0.7

    # Example buffer function (adjust as needed based on your specific buffering requirements)
    def buffer_points(self, data_array, buffer_size):
        """
        Buffers a 2D array by replacing values within a specified distance
        from non-zero values with the same non-zero values.

        Args:
            data_array (numpy.ndarray): 2D array representing the data.
            buffer_size (int): Size of the buffer in grid cells.

        Returns:
            numpy.ndarray: The buffered 2D array.
        """

        import numpy as np
        from scipy.ndimage import distance_transform_edt

        # Find the coordinates of all non-zero points
        non_zero_coords = np.where(data_array != 0)
        buffered_array = np.zeros_like(data_array)

        # Calculate the distance transform
        distances = distance_transform_edt(data_array == 0)

        # Iterate through the non-zero points and apply buffering
        for i in range(len(non_zero_coords[0])):
          row = non_zero_coords[0][i]
          col = non_zero_coords[1][i]

          # Find all points within the buffer distance
          buffer_mask = distances <= buffer_size
          buffered_array[buffer_mask] = data_array[non_zero_coords[0][i], non_zero_coords[1][i]]

        return buffered_array

    def calculate_dewpoint(self, dset, level, time):
      """Calculates the dewpoint temperature from relative humidity and temperature."""

      # Replace 't' and 'r' with the actual variable names for temperature and relative humidity in your dataset.
      temperature = dset.t.loc[time, level, :, :]
      relative_humidity = dset.r.loc[time, level, :, :]
      # Ensure units are correct for metpy calculations
      temperature = temperature * units.kelvin
      relative_humidity = relative_humidity * units.percent
      dewpoint = mpcalc.dewpoint_from_relative_humidity(temperature, relative_humidity)

      return dewpoint

    def advRelVort(self,dset,time,level_Pa=500):

        u = np.array(dset.u.loc[time,level_Pa,:,:])#*units('m/s')
        v = np.array(dset.v.loc[time,level_Pa,:,:])#*units('m/s')

        rel_vort = mg.hcurl(u, v, dset.latitude, dset.longitude)
        rel_vort = 100000*rel_vort

        adv_rel_vort = mg.hadv(u, v, rel_vort, dset.latitude, dset.longitude)
        #adv_rel_vort = adv_rel_vort #remove NaN from the boundary

        # self.calculate_absvort(dset,level_Pa)
        # vort_adv = mpcalc.advection(AbsVort, [u, v], (dx, dy), dim_order='yx') * 1e9
        return adv_rel_vort

    def advTemp(self,dset,level_Pa,time):

        u = np.array(dset.u.loc[time,level_Pa,:,:])#*units('m/s')
        v = np.array(dset.v.loc[time,level_Pa,:,:])#*units('m/s')
        t = np.array(dset.t.loc[time,level_Pa,:,:])

        adv_temp = mg.hadv(u, v, t, dset.latitude, dset.longitude)

        return adv_temp

    def div(self,dset,level_Pa,time):

        u = np.array(dset.u.loc[time,level_Pa,:,:])#*units('m/s')
        v = np.array(dset.v.loc[time,level_Pa,:,:])#*units('m/s')

        div = mg.hdivg(u,v,dset.latitude,dset.longitude)

        # dx = mpcalc.lat_lon_grid_deltas(grib.longitude, grib.latitude)
        # div = mpcalc.divergence(u * units('m/s'), v * units('m/s'), dx[1], dx[0])

        return div

    def index_k(self,dset,time):

        td850 = np.array(Calc().calculate_dewpoint(dset, 850, time))
        td700 = np.array(Calc().calculate_dewpoint(dset, 700, time))
        t500 = np.array(dset.t.loc[time,500,:,:]-273.15)
        t850 = np.array(dset.t.loc[time,850,:,:]-273.15)
        Dep700 = np.array(dset.t.loc[time,700,:,:]-273.15) - td700
        K = t850 - t500 + td850 - Dep700
        return K

    def interp(self, dset, new_lat, new_lon, method):
        'dset   = Dataset  \
        new_lat = array    \
        new_lon = array    \
        method  = str {“linear”, “nearest”} for multidimensional array,  \
                  {“linear”, “nearest”, “zero”, “slinear”, “quadratic”, “cubic”} for 1-dimensional array.'

        dsi = dset.interp(lat=new_lat, lon=new_lon, method=method)
        return dsi

    def lift_index(self,dset,time):

      lift_index = np.array(dset['lftx'].loc[time,:,:])

      return lift_index

    def precAccum24(self,dset):

      acumulados = grib2.tprate.groupby('time.day').sum()

      dias = np.array(acumulados.day)
      precipitacoes = [np.array(acumulados.loc[x,:,:]*10000) for x in dias] # x3600 se ficar estranho

      label = pd.date_range(start=f'{ano}-{mes}-{dia}', periods=5, freq='1d')

      return label,precipitacoes


    def relvort(self,dset,time,level_Pa=925):

        u = np.array(dset.u.loc[time,level_Pa,:,:])#*units('m/s')
        v = np.array(dset.v.loc[time,level_Pa,:,:])#*units('m/s')

        rel_vort = mg.hcurl(u, v, dset.latitude, dset.longitude)
        rel_vort = 100000*rel_vort

        return rel_vort

    def specific_humidity(self,dset,level_Pa,time):
        es=6.107*np.exp(17.6153*(dset.t.loc[time,level_Pa,:,:]-273.15)/
                        ((dset.t.loc[time,level_Pa,:,:]-273.15)+242.567))

        e=dset.r.loc[time,level_Pa,:,:]*es/100
        Spcf_Hmd=e*622.0/((level_Pa)-e)
        return Spcf_Hmd

    def streamlines(self,dset,level_Pa,time):
      #'time': 40, 'isobaricInhPa': 33, 'latitude': 161, 'longitude': 169
        uvel, lonu = add_cyclic_point(np.array(dset.u.loc[time,level_Pa,:,:]), coord=dset.longitude)
        vvel, lonv = add_cyclic_point(np.array(dset.v.loc[time,level_Pa,:,:]), coord=dset.longitude)
        lat_stream = dset.latitude
        lonu = np.where(lonu>=180.,lonu-360.,lonu)
        return lonu, lat_stream, uvel, vvel

    def totaltotals(self,dset,time):
        t500 = np.array(dset.t.loc[time,500,:,:]-273.15)
        t850 = np.array(dset.t.loc[time,850,:,:]-273.15)
        td850 = np.array(Calc().calculate_dewpoint(dset, 850, time))
        tts = ( (t850 + td850) - 2 * t500 )
        return tts

    def thickness(self,dset,time):
        tmed = (dset.t.loc[time,1000.,:,:]+
                dset.t.loc[time,500.,:,:])/2
        thickness = ((287*tmed)/9.8)*np.log(1000/500)
        return np.array(thickness/10)

    def wnd_spd(self,dset,level_Pa,time):
        wnd_spd = np.sqrt((np.power(np.array(dset.u.loc[time,level_Pa,:,:]),2)+
                           np.power(np.array(dset.v.loc[time,level_Pa,:,:]),2)))
        return wnd_spd

**Código 18 / 221 / 45 / 103**

In [ ]:
from matplotlib.cm import get_cmap
import matplotlib
class Plot(Calc):
  """

  Classe para plotar gráficos com base nas funções da classe Calc

  """
  def __init__(self):
      super().__init__()

  ###################################################
  # Data de inicialização e hora de previsão
  ###################################################

  def date_info(self, dset):
      global initial_time,forecast_time,dateTimeObj
      #mmddyyyy
      initial_time = pd.to_datetime(dset.coords['time'].to_pandas())

      #timedelta
      forecast_time = pd.to_timedelta(dset.coords['step'].to_pandas())

      #daterange
      dateTimeObj = initial_time  + forecast_time

      return initial_time, forecast_time, dateTimeObj

  ###################################################
  # Configura linhas de costa e dos estados
  ###################################################

  def base_of_the_fig(self,ax, dset,proj):
      ax.set_extent([float(dset.longitude.min())-360, float(dset.longitude.max())-360,float(dset.latitude.min()),
                      float(dset.latitude.max())], proj)

      ax.add_feature(ft.COASTLINE, linewidth=1.3, edgecolor='k')
      ax.add_feature(ft.BORDERS, linewidth=1.3, edgecolor='k')

      # Adicionar estados do Brasil
      states = ft.NaturalEarthFeature(category='cultural', scale='10m', facecolor='none',
                                          name='admin_1_states_provinces_lines')
      ax.add_feature(states, linewidth=1.3, edgecolor='k')

  ##########################################################
  # Configura linhas de costa e dos estados - AMERICA DO SUL
  ##########################################################

  def america_do_sul(self,ax, dset,proj):
      ax.set_extent([-120, 0, -60, 20], proj)

      ax.add_feature(ft.COASTLINE, linewidth=1.5, edgecolor='k')
      ax.add_feature(ft.BORDERS, linewidth=1.5, edgecolor='k')

      # Adicionar estados do Brasil
      states = ft.NaturalEarthFeature(category='cultural', scale='10m', facecolor='none',
                                          name='admin_1_states_provinces_lines')
      ax.add_feature(states, linewidth=1.5, edgecolor='k')

  ###################################################
  # Configura linhas de costa e dos estados
  ###################################################

  def sudeste(self,ax, dset,proj):
      ax.set_extent([-52,-36,-32,-14], proj)

      ax.add_feature(ft.COASTLINE, linewidth=1.5, edgecolor='k')
      ax.add_feature(ft.BORDERS, linewidth=1.5, edgecolor='k')

      # Adicionar estados do Brasil
      states = ft.NaturalEarthFeature(category='cultural', scale='10m', facecolor='none',
                                          name='admin_1_states_provinces_lines')
      ax.add_feature(states, linewidth=1.5, edgecolor='k')


  ###################################################
  # Ajusta eixos da figura
  ###################################################
  def axis_fig(self,ax,dset):

      x_lon = np.arange(float(np.min(dset.longitude))-360, float(np.max(dset.longitude))-360, 10)
      ax.set_xticks(x_lon, minor=False, crs=ccrs.PlateCarree(), )
      ax.set_xticklabels(ax.get_xticks(), fontsize=self.size)
      lon_formatter = LongitudeFormatter(zero_direction_label=True)
      ax.xaxis.set_major_formatter(lon_formatter)

      y_lat = np.arange(np.min(dset.latitude), np.max(dset.latitude)+1, 10)
      ax.set_yticks(y_lat, minor=False, crs=ccrs.PlateCarree())
      ax.set_yticklabels(ax.get_yticks(), fontsize=self.size)
      lat_formatter = LatitudeFormatter()
      ax.yaxis.set_major_formatter(lat_formatter)

  ###################################################
  # Ajusta eixos da figura - AMÉRICA DO SUL
  ###################################################
  def axis_AS(self,ax,dset):

      x_lon = np.arange(-120, 0, 10)
      ax.set_xticks(x_lon, minor=False, crs=ccrs.PlateCarree(), )
      ax.set_xticklabels(ax.get_xticks(), fontsize=self.size)
      lon_formatter = LongitudeFormatter(zero_direction_label=True)
      ax.xaxis.set_major_formatter(lon_formatter)

      y_lat = np.arange(-60, 20+1, 10)
      ax.set_yticks(y_lat, minor=False, crs=ccrs.PlateCarree())
      ax.set_yticklabels(ax.get_yticks(), fontsize=self.size)
      lat_formatter = LatitudeFormatter()
      ax.yaxis.set_major_formatter(lat_formatter)

  ###################################################
  # Ajusta eixos da figura - SUDESTE
  ###################################################
  def axis_sudeste(self,ax,dset):

      x_lon = np.arange(-52, -36, 2)
      ax.set_xticks(x_lon, minor=False, crs=ccrs.PlateCarree(), )
      ax.set_xticklabels(ax.get_xticks(), fontsize=self.size)
      lon_formatter = LongitudeFormatter(zero_direction_label=True)
      ax.xaxis.set_major_formatter(lon_formatter)

      y_lat = np.arange(-32, -14, 2)
      ax.set_yticks(y_lat, minor=False, crs=ccrs.PlateCarree())
      ax.set_yticklabels(ax.get_yticks(), fontsize=self.size)
      lat_formatter = LatitudeFormatter()
      ax.yaxis.set_major_formatter(lat_formatter)


  def set_initial_time(self, ax,dset,initial_time):
      ax.set_title(f'RODADA DE: {initial_time.strftime("%d/%m/%Y %H UTC")}',
                   fontsize=self.size, loc='left')

  ###################################################
  # Salva a figura
  ###################################################

  def save_figure(self,fig_name):
      try:
          os.mkdir('figures/')
          plt.savefig(f'figures/{fig_name}.png',
              bbox_inches='tight', dpi=70, facecolor='w', edgecolor='w')

      except:

            plt.savefig(f'figures/{fig_name}.png',
              bbox_inches='tight', dpi=70, facecolor='w', edgecolor='w')

      plt.close()

  #########################################################################################################################################################
  # GPT - 18 (UMIDADE E/OU NEBULOSIDADE)
  #########################################################################################################################################################

  def gpt18_umidade(self,dset,dset1,time):
      initial_time, forecast_time, dateTimeObj = self.date_info(dset)
      initial_time = initial_time[0]

      plt.figure(figsize=(10,10))
      proj  =  ccrs.PlateCarree()
      ax = plt.axes(projection=proj)
      self.america_do_sul(ax, dset, proj)
      self.axis_AS(ax, dset)

      ################################
      # Prepara u e v do vento > 5 m/s
      ################################
      U = np.array(dset.u.loc[time,1000,::self.step,::self.step])
      V = np.array(dset.v.loc[time,1000,::self.step,::self.step])

      # Fazer o filtro depois da velocidade > 5m/s
      #u = np.where(u > 5, u, 0)
      #v = np.where(v > 5, v, 0)

      #############################
      # Prepara variável da umidade
      #############################
      UMIDADE_850 = np.array(dset.r.loc[time,850,:,:])
      UMIDADE_700 = np.array(dset.r.loc[time,700,:,:])
      UMIDADE_500 = np.array(dset.r.loc[time,500,:,:])
      UMIDADE_1000 = np.array(dset.r.loc[time,1000,:,:])

      umidade     = np.mean([UMIDADE_850,UMIDADE_700,UMIDADE_500],axis=0)
      umidade_sup = np.mean([UMIDADE_850,UMIDADE_1000],axis=0)

      ###############################
      # Prepara variável da espessura
      ###############################
      espessura = self.thickness(dset,time)

      ####################
      # Plota umidade > 60
      ####################
      clevs1 = np.arange(60,101,10)

      plt.contourf(dset.longitude, dset.latitude,
                    umidade,
                    cmap='Blues',
                    levels=clevs1,
                    transform=ccrs.PlateCarree(),
                    extend = None)

      ##########################
      # Plota linha de espessura
      ##########################
      line = plt.contour(dset.longitude, dset.latitude,
                  espessura,
                  colors ='red',
                  transform=ccrs.PlateCarree(),
                  levels = np.arange(540,595,6),
                  extend = None)

      idline = ax.clabel(line, inline=True, fontsize=10, fmt='%d')

      ########################################
      # Plota linha de umidade superficie > 60
      ########################################
      line1 = plt.contour(dset.longitude, dset.latitude,
                  umidade_sup,
                  colors ='green',
                  transform=ccrs.PlateCarree(),
                  levels = np.arange(60,101,20),
                  extend = None)

      idline1 = ax.clabel(line1, inline=True, fontsize=10, fmt='%d')

      ######################################
      # Plota linha de pressão em superficie
      ######################################
      line2 = plt.contour(dset.longitude, dset.latitude,
                          dset1.msl.loc[time,:,:]/100,
                          colors ='black',
                          levels = np.arange(969,1030,3),
                          linewidths= self.linewidth)

      idline2 = ax.clabel(line2, inline=True, fontsize=10,fmt='%d')

      #############################
      # Plota o vento em superficie
      #############################
      plt.quiver(dset.longitude[::self.step], dset.latitude[::self.step],
                U,
                V,
                color='k',
                transform=proj)

      ax.set_title(f'UMIDADE E/OU NEBULOSIDADE {hora_prev}z',
                   fontsize=self.size, loc='center')

      #ax.text(-76, -40,'URM>60(850/700/500)+URM>60(1000/850)(cont) + VENTO>5m/s+ESP(500/1000)',fontsize=7)
      ax.text(-73, -65,'URM>60(850/700/500)+URM>60(1000/850)(cont) + ESP(500/1000)',fontsize=7)

      self.date_info(dset)
      ax.set_title(f'VÁLIDO EM: {time}'[:-13]+' UTC', fontsize=self.size, loc='right')

      self.set_initial_time(ax,dset,initial_time)
      self.save_figure(f'gpt18_umidade_{time}.png')
      plt.show()


  #########################################################################################################################################################
  # GPT - 18 (PANCADAS POR CALOR E UMIDADE)
  #########################################################################################################################################################
  def gpt18_pancadas(self,dset,dset1,time):
      initial_time, forecast_time, dateTimeObj = self.date_info(dset)
      initial_time = initial_time[0]

      plt.figure(figsize=(10,10))
      proj  =  ccrs.PlateCarree()
      ax = plt.axes(projection=proj)
      self.america_do_sul(ax, dset, proj)
      self.axis_AS(ax, dset)

      #########################################
      # Prepara variável da altura geopotencial
      #########################################
      HGT_300 = np.array(dset.gh.loc[time,300,:,:])

      ##############################
      # Prepara variável do Índice K
      ##############################
      K = Calc().index_k(dset,time)

      ################################
      # Prepara variável do Índice TTS
      ################################
      TTS = Calc().totaltotals(dset,time)

      ###############################
      # Prepara variável do Índice LI
      ###############################
      LIFT = Calc().lift_index(dset1,time)

      #########
      # Filtros
      #########
      FILTRO_K    = np.where(K>30, 1, 0)
      FILTRO_TTS  = np.where(TTS>45 , 1, 0)
      FILTRO_LIFT = np.where(LIFT<-1 , 1, 0)

      # VERDE       = np.where((FILTRO_K==1) & (FILTRO_TTS == 1), 1, 0)
      # VERMELHO    = np.where( (FILTRO_LIFT == 1) & (VERDE==1), 1, 0)

      #################################
      # Plota K > 30 + TTS > 45 (verde)
      #################################
      # PREPARA A COR DO CONTOURF
      verde = sequential_hcl(h = 135, c = 45, l = [40, 95], power = 1.3, rev = True)

      # Plota somente valores onde VERDE == 1
      # plt.contourf(grib.longitude, grib.latitude, VERDE,
      #              transform=ccrs.PlateCarree(),
      #              colors=verde.colors(1),
      #              levels=[0.1,2])

      plt.contourf(grib.longitude, grib.latitude, VERDE,
                   transform=ccrs.PlateCarree(),
                   cmap='Greens',
                   levels=np.arange(np.min(VERDE),np.max(VERDE),2))

      #################################
      # Plota K > 30 + TTS > 45 (vermelho)
      #################################
      # Plota somente valores onde VERMELHO == 1
      vermelho = sequential_hcl(h = 10, c = 85, l = [40, 65], power = 1.3, rev = True)

      # plt.contourf(grib.longitude, grib.latitude, VERMELHO,
      #              transform=ccrs.PlateCarree(),
      #              colors=vermelho.colors(1),
      #              levels=[0.1,2])

      plt.contourf(grib.longitude, grib.latitude, VERMELHO,
                   transform=ccrs.PlateCarree(),
                   cmap='Reds',
                   levels=np.arange(np.min(VERMELHO),-2))

      ####################################
      # Plota linha de altura geopotencial
      ####################################
      line = plt.contour(dset.longitude, dset.latitude,
                  HGT_300,
                  colors ='black',
                  transform=ccrs.PlateCarree(),
                  levels = np.arange(np.min(HGT_300),np.max(HGT_300),80),
                  extend = None)

      idline = ax.clabel(line, inline=True, fontsize=10, fmt='%d')

      ###########################
      # Plota linha do LIFT INDEX
      ###########################
      line1 = plt.contour(dset.longitude, dset.latitude,
                          LIFT,
                          colors ='blue',
                          levels = np.arange(np.min(LIFT),-1,2),
                          linewidths= self.linewidth)

      idline1 = ax.clabel(line1, inline=True, fontsize=10,fmt='%d')

      ax.set_title(f'PANCADAS POR CALOR E UMIDADE {hora_prev}z',
                   fontsize=self.size, loc='center')

      ax.text(-73, -40,'[K>30+TTS>45](verde)+[K>30+TTS>45+LI<-1](vermelho)+LIF(cont.)+HGT(300)',fontsize=7)

      self.date_info(dset)
      ax.set_title(f'VÁLIDO EM: {time}'[:-13]+' UTC', fontsize=self.size, loc='right')

      self.set_initial_time(ax,dset,initial_time)
      self.save_figure(f'gpt18_pancadas_{time}')
      plt.show()


  #########################################################################################################################################################
  # GPT - 18 (CHUVA)
  #########################################################################################################################################################
  def gpt18_chuva(self,dset, time):
      initial_time, forecast_time, dateTimeObj = self.date_info(dset)
      initial_time = initial_time[0]

      plt.figure(figsize=(5,5))
      proj  =  ccrs.PlateCarree()
      ax = plt.axes(projection=proj)
      self.america_do_sul(ax, dset, proj)
      self.axis_AS(ax, dset)

      #####################
      # ALTURA GEOPOTENCIAL
      #####################
      HGT_500 = np.array(dset.gh.loc[time,500,:,:])


      # ##########
      # # UMIDADE
      # ##########
      UMIDADE_850 = np.array(dset.r.loc[time,850,:,:])
      UMIDADE_700 = np.array(dset.r.loc[time,700,:,:])
      UMIDADE_500 = np.array(dset.r.loc[time,500,:,:])
      umidade = np.mean([UMIDADE_850,UMIDADE_700,UMIDADE_500],axis=0)

      # Filtros
      FILTRO_UMIDADE   = np.where(umidade>40 , umidade, 0)
      FILTRO_UMIDADE_1 = np.where(umidade>70 , umidade, 0)

      # #################
      # # omega em 500hPa
      # #################
      OMEGA = dset.w.loc[time,500,:,:]

      # Filtros
      FILTRO_OMEGA     = np.where((OMEGA > -0.001) & (OMEGA < 0), OMEGA, 0)
      FILTRO_OMEGA_1   = np.where((OMEGA <= -0.3), OMEGA, 0)

      VERMELHO = np.where( (FILTRO_OMEGA==1) & (FILTRO_UMIDADE == 1), 1, 0)
      AZUL     = np.where((FILTRO_UMIDADE_1 == 1) & (FILTRO_OMEGA==1), 1, 0)
      VERDE    = np.where((FILTRO_UMIDADE_1 == 1) & (FILTRO_OMEGA_1==1), 1, 0)

      VERMELHO = Calc().buffer_points(VERMELHO,2)
      AZUL     = Calc().buffer_points(AZUL,2)
      VERDE    = Calc().buffer_points(VERDE,2)


      # PREPARA A COR DO CONTOURF
      verde    = sequential_hcl(h = 135, c = 45, l = [60, 95], power = 1.3, rev = True)
      vermelho = sequential_hcl(h = 10, c = 85, l = [40, 65], power = 1.3, rev = True)
      azul     = sequential_hcl(h = 270, c = 70, l = [30, 95], power = 1.2)

      # Plota somente valores onde VERMELHO == 1
      plt.contourf(grib.longitude, grib.latitude, VERMELHO, transform=ccrs.PlateCarree(), colors=vermelho.colors(1),  levels=[0.1,2])

      # Plota somente valores onde AZUL == 1
      plt.contourf(grib.longitude, grib.latitude, AZUL, transform=ccrs.PlateCarree(), colors=azul.colors(1),  levels=[0.1,2])

      # Plota somente valores onde VERDE == 1
      plt.contourf(grib.longitude, grib.latitude, VERDE, transform=ccrs.PlateCarree(), colors=verde.colors(1), levels=[0.1,2])

      # Plota linha de corrente em 500hPa
      lonu, lat_stream, uvel, vvel= self.streamlines(dset,500,time)

      ax.streamplot(lonu, lat_stream ,uvel, vvel,linewidth=0.5,
            arrowsize = .8, density=2, color='k',
            transform=ccrs.PlateCarree() )

      ####################################
      # Plota linha de altura geopotencial
      ####################################
      line = plt.contour(dset.longitude, dset.latitude,
                  HGT_500,
                  colors ='black',
                  transform=ccrs.PlateCarree(),
                  levels = np.arange(np.min(HGT_500),np.max(HGT_500),30),
                  extend = None)

      idline = ax.clabel(line, inline=True, fontsize=10, fmt='%d')


      ax.set_title(f'CHUVA {hora_prev}z',
                   fontsize=self.size, loc='center')

      self.date_info(dset)
      ax.set_title(f'VÁLIDO EM: {time}'[:-13]+' UTC', fontsize=self.size, loc='right')

      ax.text(-78, -40,'[OMEGA(-0.001)(azul)+OMEGA(-0.3)(verde)_URM>70(850/700/500)] + STREAMLINE(500) \
      \n                                   [OMEGA(-0.001)_URM>40(850/750/500)(vermelho)]',fontsize=7)

      self.set_initial_time(ax,dset,initial_time)
      self.save_figure(f'gpt18_chuva_{time}')
      plt.show()


  #########################################################################################################################################################
  # GPT - 221 (CAPE OMEGA DO ETA 40KM E GFS)
  #########################################################################################################################################################
  def gpt221_brasil(self,dset,dset2,time):
      initial_time, forecast_time, dateTimeObj = self.date_info(dset)
      initial_time = initial_time[0]

      plt.figure(figsize=(10,10))
      proj  =  ccrs.PlateCarree()
      ax = plt.axes(projection=proj)
      self.america_do_sul(ax, dset, proj)
      self.axis_AS(ax, dset)


      ################################
      # Prepara u e v do vento > 5 m/s
      ################################
      U = np.array(dset.u.loc[time,1000,::self.step,::self.step])
      V = np.array(dset.v.loc[time,1000,::self.step,::self.step])

      #######
      # CAPE
      #######
      CAPE = np.array(dset2.mucape.loc[time,:,:])

      # ##########
      # # UMIDADE
      # ##########

      UMIDADE_1000 = np.array(dset.r.loc[time,1000,:,:])
      UMIDADE_850  = np.array(dset.r.loc[time,850,:,:])
      UMIDADE_700  = np.array(dset.r.loc[time,700,:,:])
      UMIDADE_500  = np.array(dset.r.loc[time,500,:,:])
      UMIDADE_400  = np.array(dset.r.loc[time,400,:,:])
      UMIDADE_200  = np.array(dset.r.loc[time,200,:,:])

      umidade     = np.mean([UMIDADE_850,UMIDADE_700,UMIDADE_500],axis=0)
      umidade_sup = np.mean([UMIDADE_850,UMIDADE_1000],axis=0)
      umidade_top = np.mean([UMIDADE_400,UMIDADE_200],axis=0)

      # Filtros
      FILTRO_UMIDADE   = np.where(umidade>60 , umidade, 0)
      FILTRO_UMIDADE_1 = np.where(umidade<40 , umidade, 0)
      FILTRO_UMIDADE_2 = np.where(umidade_sup>60 , umidade_sup, 0)

      FILTRO_UMIDADE_3 = np.where(umidade_sup<40 , umidade_sup, 0)
      FILTRO_UMIDADE_4 = np.where(umidade<40 , umidade, 0)

      FILTRO_UMIDADE_5 = np.where( (FILTRO_UMIDADE_3 != 0) & (FILTRO_UMIDADE_4 != 0) , np.mean([FILTRO_UMIDADE_3,FILTRO_UMIDADE_4],axis=0), 0)

      FILTRO_UMIDADE_6 = np.where(umidade_top>60 , umidade_top, 0)

      # #################
      # # omega em 500hPa
      # #################
      OMEGA = dset.w.loc[time,500,:,:]*10

      #####################################
      # URM((850/700/500)/3)>60 # plot azul
      #####################################
      plt.contourf(grib.longitude, grib.latitude, FILTRO_UMIDADE,
                   transform=ccrs.PlateCarree(),
                   cmap='Blues',
                   levels=np.arange(60,101,10))

      ###############################################################
      # URM((850/700/500)/3)<40 e URM((1000/850)/2)<40 # plot laranja
      ###############################################################
      plt.contourf(grib.longitude, grib.latitude, FILTRO_UMIDADE_5,
                   transform=ccrs.PlateCarree(),
                   cmap='Oranges_r',
                   levels=np.arange(.01,40,1))

      #######################################
      #  URM((1000/850)/2)>60 # contour verde
      #######################################
      line = plt.contour(dset.longitude, dset.latitude,
                  FILTRO_UMIDADE_2,
                  colors ='green',
                  transform=ccrs.PlateCarree(),
                  levels = np.arange(60,100,20),
                  extend = None,
                  linewidths=.8)

      #######################################
      #  URM((400/200)/2)>60 # contour roxo
      #######################################
      line = plt.contour(dset.longitude, dset.latitude,
                  FILTRO_UMIDADE_6,
                  colors ='purple',
                  transform=ccrs.PlateCarree(),
                  levels = np.arange(60,100,10),
                  extend = None,
                  linewidths=.8)

      #######################################
      #  CAPE/100
      #######################################
      line = plt.contour(dset.longitude, dset.latitude,
                  CAPE/100,
                  colors ='Red',
                  transform=ccrs.PlateCarree(),
                  levels = np.arange(5,np.max(CAPE),10),
                  extend = None,
                  linewidths=.8)

      #######################################
      # omega em 500hPa
      #######################################
      line = plt.contour(dset.longitude, dset.latitude,
                  OMEGA,
                  colors ='k',
                  transform=ccrs.PlateCarree(),
                  levels = np.arange(np.min(OMEGA),-0.3,0.8),
                  extend = None,
                  linewidths=1)

      #############################
      # Plota o vento em superficie
      #############################
      plt.quiver(dset.longitude[::self.step], dset.latitude[::self.step],
                U,
                V,
                color='m',
                transform=proj)

      ax.set_title(f'GPT 221 - BRASIL',
                   fontsize=self.size, loc='center')

      self.date_info(dset)
      ax.set_title(f'VÁLIDO EM: {time}'[:-13]+' UTC', fontsize=self.size, loc='right')

      ax.text(-78, -67,'URM((850/700/500)/3)>60+URM((1000/850)/2)>60+{URM(850/700/500)/3)<40_URM(1000/850)/2)<40} \n +URM((400/200)/2)>60+CAPE/100+OMEGA(500)*10+VENTO(sup)',fontsize=7)

      self.set_initial_time(ax,dset,initial_time)
      self.save_figure(f'gpt221_brasil_{time}')
      plt.show()

  #########################################################################################################################################################
  # GPT - 221 (CAPE OMEGA DO ETA 40KM E GFS)
  #########################################################################################################################################################
  def gpt221_sudeste(self,dset,dset2,time):
      initial_time, forecast_time, dateTimeObj = self.date_info(dset)
      initial_time = initial_time[0]

      plt.figure(figsize=(10,10))
      proj  =  ccrs.PlateCarree()
      ax = plt.axes(projection=proj)
      self.sudeste(ax, dset, proj)
      self.axis_sudeste(ax, dset)


      ################################
      # Prepara u e v do vento > 5 m/s
      ################################
      U = np.array(dset.u.loc[time,1000,::self.step,::self.step])
      V = np.array(dset.v.loc[time,1000,::self.step,::self.step])

      #######
      # CAPE
      #######
      CAPE = np.array(dset2.mucape.loc[time,:,:])

      # ##########
      # # UMIDADE
      # ##########
      UMIDADE_1000 = np.array(dset.r.loc[time,1000,:,:])
      UMIDADE_850  = np.array(dset.r.loc[time,850,:,:])
      UMIDADE_700  = np.array(dset.r.loc[time,700,:,:])
      UMIDADE_500  = np.array(dset.r.loc[time,500,:,:])
      UMIDADE_400  = np.array(dset.r.loc[time,400,:,:])
      UMIDADE_200  = np.array(dset.r.loc[time,200,:,:])

      umidade     = np.mean([UMIDADE_850,UMIDADE_700,UMIDADE_500],axis=0)
      umidade_sup = np.mean([UMIDADE_850,UMIDADE_1000],axis=0)
      umidade_top = np.mean([UMIDADE_400,UMIDADE_200],axis=0)

      # Filtros
      FILTRO_UMIDADE   = np.where(umidade>60 , umidade, 0)
      FILTRO_UMIDADE_1 = np.where(umidade<40 , umidade, 0)
      FILTRO_UMIDADE_2 = np.where(umidade_sup>60 , umidade_sup, 0)

      FILTRO_UMIDADE_3 = np.where(umidade_sup<40 , umidade_sup, 0)
      FILTRO_UMIDADE_4 = np.where(umidade<40 , umidade, 0)

      FILTRO_UMIDADE_5 = np.where( (FILTRO_UMIDADE_3 != 0) & (FILTRO_UMIDADE_4 != 0) , np.mean([FILTRO_UMIDADE_3,FILTRO_UMIDADE_4],axis=0), 0)

      FILTRO_UMIDADE_6 = np.where(umidade_top>60 , umidade_top, 0)

      # #################
      # # omega em 500hPa
      # #################
      OMEGA = dset.w.loc[time,500,:,:]*10

      #####################################
      # URM((850/700/500)/3)>60 # plot azul
      #####################################
      plt.contourf(grib.longitude, grib.latitude, FILTRO_UMIDADE,
                   transform=ccrs.PlateCarree(),
                   cmap='Blues',
                   levels=np.arange(60,101,10))

      ###############################################################
      # URM((850/700/500)/3)<40 e URM((1000/850)/2)<40 # plot laranja
      ###############################################################
      plt.contourf(grib.longitude, grib.latitude, FILTRO_UMIDADE_5,
                   transform=ccrs.PlateCarree(),
                   cmap='Oranges_r',
                   levels=np.arange(.01,40,1))

      #######################################
      #  URM((1000/850)/2)>60 # contour verde
      #######################################
      line = plt.contour(dset.longitude, dset.latitude,
                  FILTRO_UMIDADE_2,
                  colors ='green',
                  transform=ccrs.PlateCarree(),
                  levels = np.arange(60,100,20),
                  extend = None,
                  linewidths=.8)

      #######################################
      #  URM((400/200)/2)>60 # contour roxo
      #######################################
      line = plt.contour(dset.longitude, dset.latitude,
                  FILTRO_UMIDADE_6,
                  colors ='purple',
                  transform=ccrs.PlateCarree(),
                  levels = np.arange(60,100,10),
                  extend = None,
                  linewidths=.8)

      #######################################
      #  CAPE/100
      #######################################
      line = plt.contour(dset.longitude, dset.latitude,
                  CAPE/100,
                  colors ='Red',
                  transform=ccrs.PlateCarree(),
                  levels = np.arange(5,np.max(CAPE),10),
                  extend = None,
                  linewidths=.8)

      #######################################
      # omega em 500hPa
      #######################################
      line = plt.contour(dset.longitude, dset.latitude,
                  OMEGA,
                  colors ='k',
                  transform=ccrs.PlateCarree(),
                  levels = np.arange(np.min(OMEGA),-0.3,0.3),
                  extend = None,
                  linewidths=1)

      #############################
      # Plota o vento em superficie
      #############################
      plt.quiver(dset.longitude[::self.step], dset.latitude[::self.step],
                U,
                V,
                color='m',
                transform=proj)

      ax.set_title(f'GPT 221 - SUDESTE',
                   fontsize=self.size, loc='center')

      self.date_info(dset)
      ax.set_title(f'VÁLIDO EM: {time}'[:-13]+' UTC', fontsize=self.size, loc='right')

      ax.text(-54, -34,'URM((850/700/500)/3)>60+URM((1000/850)/2)>60+{URM(850/700/500)/3)<40_URM(1000/850)/2)<40} \n +URM((400/200)/2)>60+CAPE/100+OMEGA(500)*10+VENTO(sup)',fontsize=7)

      self.set_initial_time(ax,dset,initial_time)
      self.save_figure(f'gpt221_sudeste_{time}')
      plt.show()


  #########################################################################################################################################################
  # GPT - 45 (CHUVA CONTINUA)
  #########################################################################################################################################################
  def gpt45_sudeste(self,dset,dset2,dset3,time):
      initial_time, forecast_time, dateTimeObj = self.date_info(dset)
      initial_time = initial_time[0]

      plt.figure(figsize=(15,15))
      proj  =  ccrs.PlateCarree()
      ax = plt.axes(projection=proj)
      # self.base_of_the_fig(ax, dset, proj)
      # self.axis_fig(ax, dset)

      self.sudeste(ax, dset, proj)
      self.axis_sudeste(ax, dset)

      ###################
      # AGUA PRECIPITAVEL
      ###################
      pwat = np.array(dset2.tcwv.loc[time,:,:])

      ###############
      # OMEGA 850 hPa
      ###############
      OMEGA_850 = dset.w.loc[time,850,:,:]*10

      ###############
      # OMEGA 500 hPa
      ###############
      OMEGA_500 = dset.w.loc[time,500,:,:]*10

      #######
      # msl
      #######
      msl = dset3.msl.loc[time,:,:]/100

      ##########
      # UMIDADE
      ##########
      UMIDADE_100 = np.array(dset.r.loc[time,100,:,:])
      UMIDADE_850  = np.array(dset.r.loc[time,850,:,:])

      UMIDADE_UP    = np.mean([UMIDADE_850,UMIDADE_100],axis=0)
      FILTRO_UMIDADE = np.where(UMIDADE_UP>80 , UMIDADE_UP, 0)

      ################################
      # Prepara u e v do vento > 5 m/s
      ################################
      U = np.array(dset.u.loc[time,1000,::self.step,::self.step])
      V = np.array(dset.v.loc[time,1000,::self.step,::self.step])

      ###################
      # AGUA PRECIPITAVEL
      ###################
      cb = plt.contourf(grib.longitude, grib.latitude, pwat,
                 transform=ccrs.PlateCarree(),
                 cmap='Greens',
                 levels=np.arange(40,65,5),
                 extend='max')

      plt.colorbar(cb,ax=ax,orientation='vertical',pad=0, shrink=0.2)

      ###################
      # UMIDADE_UP
      ###################
      # plt.contourf(grib.longitude, grib.latitude, FILTRO_UMIDADE,
      #              transform=ccrs.PlateCarree(),
      #              cmap='GnBu_r',
      #              levels=np.arange(80,np.max(FILTRO_UMIDADE),5))

      level = np.arange(80,np.max(FILTRO_UMIDADE),5)
      plt.contourf(grib.longitude, grib.latitude, FILTRO_UMIDADE,
                   transform=ccrs.PlateCarree(),
                   levels=level,
                   colors=['#10c5c2'] * len(level))

      #######################################
      # omega em 500hPa
      #######################################
      line = plt.contour(dset.longitude, dset.latitude,
                  OMEGA_500,
                  colors ='k',
                  transform=ccrs.PlateCarree(),
                  levels = np.arange(np.min(OMEGA_500),-0.2,0.2),
                  extend = None,
                  linewidths=.8)

      #######################################
      # omega em 850hPa
      #######################################
      line1 = plt.contour(dset.longitude, dset.latitude,
                  OMEGA_850,
                  colors ='red',
                  transform=ccrs.PlateCarree(),
                  levels = np.arange(np.min(OMEGA_850),-0.2,0.2),
                  extend = None,
                  linewidths=.8)

      #######################################
      # msl
      #######################################
      line2 = plt.contour(dset.longitude, dset.latitude,
                  msl,
                  colors ='k',
                  transform=ccrs.PlateCarree(),
                  levels = np.arange(np.min(msl),np.max(msl),3),
                  extend = None,
                  linewidths=1.1)

      #############################
      # Plota o vento em superficie
      #############################
      plt.quiver(dset.longitude[::self.step], dset.latitude[::self.step],
                U,
                V,
                color='k',
                transform=proj)

      idline2 = ax.clabel(line2, inline=True, fontsize=10,fmt='%d')

      ax.set_title(f'GPT 45 - CHUVA CONTÍNUA',
                   fontsize=self.size, loc='center')

      self.date_info(dset)
      ax.set_title(f'VÁLIDO EM: {time}'[:-13]+' UTC', fontsize=self.size, loc='right')

      ax.text(-52, -33,'AGPL>55(VERDE)+OMG(850)+[(VERMELHO)<(-)0.2+OMG(500)]+(PRETO)<(-)0.2 ttv>5+FGLM+[UMRL(850/1000)>60+v(qLESTE)>5m.(AZUL)]',fontsize=10)

      self.set_initial_time(ax,dset,initial_time)
      self.save_figure(f'gpt45_chuva_{time}')
      plt.show()

# 'AGPL>55(VERDE)+(PRETO)<(-)0.2 OMG(850)+[(VERMELHO)<(-)0.2+OMG(500)]+ ttv>5+FGLM+[UMRL(850/1000)>60+v(qLESTE)>5m.(AZUL)]'


  #########################################################################################################################################################
  # GPT - 103 (ZCAS - 00) - CAMPO 1
  #########################################################################################################################################################
  def gpt103_1_brasil(self,dset,dset2,dset3,time):
      initial_time, forecast_time, dateTimeObj = self.date_info(dset)
      initial_time = initial_time[0]

      plt.figure(figsize=(10,10))
      proj  =  ccrs.PlateCarree()
      ax = plt.axes(projection=proj)
      self.america_do_sul(ax, dset, proj)
      self.axis_AS(ax, dset)

      # self.sudeste(ax, dset, proj)
      # self.axis_sudeste(ax, dset)

      ###################
      # AGUA PRECIPITAVEL
      ###################
      pwat = np.array(dset2.tcwv.loc[time,:,:])

      ###############
      # OMEGA 500 hPa
      ###############
      OMEGA_500 = dset.w.loc[time,500,:,:]*10


      ################################
      # Prepara u e v do vento > 5 m/s
      ################################
      U = np.array(dset.u.loc[time,850,::self.step,::self.step])
      V = np.array(dset.v.loc[time,850,::self.step,::self.step])

      ###################
      # AGUA PRECIPITAVEL
      ###################
      colors = ('#fefefe', '#87ceeb', '#00c3ff', '#1e90ff','#006300','#fe0000','#ffd700')
      plt.contourf(grib.longitude, grib.latitude, pwat,
                    len(colors)-1,
                    colors=colors,
                    levels=[40,45,50,55,60,65],
                    transform=ccrs.PlateCarree(),
                    extend = 'both')

      cb = plt.colorbar(ax=ax, orientation="vertical", pad=0,ticks=[40,45,50,55,60,65], aspect=16, shrink=0.5)
      cb.set_label('Água precipitável (kg.m⁻²)', fontsize=self.size)
      cb.ax.tick_params(labelsize=self.size)

      #######################################
      # omega em 500hPa
      #######################################
      line = plt.contour(dset.longitude, dset.latitude,
                  OMEGA_500,
                  colors ='yellow',
                  transform=ccrs.PlateCarree(),
                  levels = np.arange(np.min(OMEGA_500),-0.2,0.2),
                  extend = None,
                  alpha  = 0.2,
                  linewidths=.8)

      #######################################
      # Plota linha de corrente em 850 hPa
      #######################################
      lonu, lat_stream, uvel, vvel= self.streamlines(dset,850,time)

      ax.streamplot(lonu, lat_stream ,uvel, vvel,linewidth=0.5,
            arrowsize = .8, density=2, color='k',
            transform=ccrs.PlateCarree() )

      ax.text(-65, -66,'LC(850)+AP>40(SOMBREADO)+OMEGA(500)',fontsize=10)

      ax.set_title('GPT 103', fontsize=self.size, loc='center')

      self.date_info(dset)
      ax.set_title(f'VÁLIDO EM: {time}'[:-13]+' UTC', fontsize=self.size, loc='right')

      self.set_initial_time(ax,dset,initial_time)
      self.save_figure(f'gpt103_1_brasil_{time}')
      plt.show()

  #########################################################################################################################################################
  # GPT - 103 (ZCAS - 00) - CAMPO 2
  #########################################################################################################################################################
  def gpt103_2_brasil(self,dset,dset2,dset3,time):
      initial_time, forecast_time, dateTimeObj = self.date_info(dset)
      initial_time = initial_time[0]

      plt.figure(figsize=(10,10))
      proj  =  ccrs.PlateCarree()
      ax = plt.axes(projection=proj)
      self.america_do_sul(ax, dset, proj)
      self.axis_AS(ax, dset)

      # self.sudeste(ax, dset, proj)
      # self.axis_sudeste(ax, dset)

      #####################
      # ALTURA GEOPOTENCIAL
      #####################
      HGT_500 = np.array(dset.gh.loc[time,500,:,:])/10

      ###############
      # OMEGA 500 hPa
      ###############
      OMEGA_500 = dset.w.loc[time,500,:,:]*10


      # ##########
      # # UMIDADE
      # ##########
      UMIDADE_500  = np.array(dset.r.loc[time,500,:,:])

      ###################
      # UMIDADE
      ###################
      cb = plt.contourf(grib.longitude, grib.latitude, UMIDADE_500,
                 transform=ccrs.PlateCarree(),
                 cmap='Greens',
                 levels=np.arange(70,100,5),
                 extend='max')

      # plt.colorbar(cb,ax=ax,orientation='horizontal', shrink=0.2)


      ####################################
      # Plota linha de altura geopotencial
      ####################################
      line = plt.contour(dset.longitude, dset.latitude,
                  HGT_500,
                  colors ='blue',
                  transform=ccrs.PlateCarree(),
                  levels = np.arange(np.min(HGT_500),np.max(HGT_500),3),
                  extend = None)

      idline = ax.clabel(line, inline=True, fontsize=10, fmt='%d')

      #######################################
      # omega em 500hPa
      #######################################
      line = plt.contour(dset.longitude, dset.latitude,
                  OMEGA_500,
                  colors ='red',
                  transform=ccrs.PlateCarree(),
                  levels = np.arange(np.min(OMEGA_500),-0.2,0.2),
                  extend = None,
                  alpha  = 0.2,
                  linewidths=.8)

      #######################################
      # Plota linha de corrente em 500 hPa
      #######################################
      lonu, lat_stream, uvel, vvel= self.streamlines(dset,500,time)

      ax.streamplot(lonu, lat_stream ,uvel, vvel,linewidth=0.5,
            arrowsize = .8, density=2, color='k',
            transform=ccrs.PlateCarree() )

      ax.text(-65, -65.5,'HGT(500)+L.CORRENTE(500)+UMIDADE(500)>70%+OMEGA(500)<0.2',fontsize=10)

      ax.set_title(f'GPT 103',
                   fontsize=self.size, loc='center')

      self.date_info(dset)
      ax.set_title(f'VÁLIDO EM: {time}'[:-13]+' UTC', fontsize=self.size, loc='right')

      self.set_initial_time(ax,dset,initial_time)
      self.save_figure(f'gpt103_2_brasil_{time}')
      plt.show()

  #########################################################################################################################################################
  # GPT - 103 (ZCAS - 00) - CAMPO 3
  #########################################################################################################################################################
  def gpt103_3_brasil(self,dset,dset2,dset3,time):
      initial_time, forecast_time, dateTimeObj = self.date_info(dset)
      initial_time = initial_time[0]

      plt.figure(figsize=(10,10))
      proj  =  ccrs.PlateCarree()
      ax = plt.axes(projection=proj)
      self.america_do_sul(ax, dset, proj)
      self.axis_AS(ax, dset)

      # self.sudeste(ax, dset, proj)
      # self.axis_sudeste(ax, dset)

      ################################
      # Prepara u e v do vento
      ################################
      U = np.array(dset.u.loc[time,250,::self.step,::self.step])
      V = np.array(dset.v.loc[time,250,::self.step,::self.step])

      jato_u = np.array(dset.u.loc[time,250,:,:])
      jato_v = np.array(dset.v.loc[time,250,:,:])
      WSPD = np.array(np.sqrt(jato_u**2 + jato_v**2)) * 1.94384 # m/s -> kt

      ################################
      # Prepara u e v do vento
      ################################
      divergencia = self.div(dset,250,time)

      #######################################
      # Plota linha de corrente em 250 hPa
      #######################################
      lonu, lat_stream, uvel, vvel= self.streamlines(dset,250,time)

      ax.streamplot(lonu, lat_stream ,uvel, vvel,linewidth=.9,
            arrowsize = .8, density=5, color='k',
            transform=ccrs.PlateCarree())


      ############################
      # Jato 250 hPa - intensidade
      ############################
      colors = ('#002323','#033939','#0c4f4f','#145959','#237878','#2e8d8d','#3da7a7')
      plt.contourf(grib.longitude, grib.latitude, WSPD,
                    len(colors)-1,
                    colors=colors,
                    levels=np.arange(70,140,10),
                    transform=ccrs.PlateCarree(),
                    extend = 'max')

      cb = plt.colorbar(ax=ax, orientation="vertical", pad=0,ticks=np.arange(70,140,10), aspect=16, shrink=0.5)
      cb.set_label('intensidade do vento (kt)', fontsize=self.size)
      cb.ax.tick_params(labelsize=self.size)

      ####################################
      # intensidade do jato - linhas
      ####################################
      line = plt.contour(dset.longitude, dset.latitude,
                  WSPD,
                  colors ='white',
                  transform=ccrs.PlateCarree(),
                  levels = np.arange(70,np.nanmax(WSPD),10),
                  extend = None)

      idline = ax.clabel(line, inline=True, fontsize=10, fmt='%d')

      ####################################
      # divergencia - linhas
      ####################################
      line = plt.contour(dset.longitude, dset.latitude,
                  divergencia,
                  colors ='red',
                  transform=ccrs.PlateCarree(),
                  levels = np.arange(10**-4, np.nanmax(divergencia), 0.25*10**-4),
                  extend = None)

      ax.text(-65, -66.5,'L.CORRENTE(250)+DIVERGÊNCIA(250)>0+JATO(250)',fontsize=10)

      ax.set_title(f'GPT 103',
                   fontsize=self.size, loc='center')

      self.date_info(dset)
      ax.set_title(f'VÁLIDO EM: {time}'[:-13]+' UTC', fontsize=self.size, loc='right')

      self.set_initial_time(ax,dset,initial_time)
      self.save_figure(f'gpt103_3_brasil_{time}')
      plt.show()

  #########################################################################################################################################################
  # GPT - 103 (ZCAS - 00) - CAMPO 4
  #########################################################################################################################################################
  def gpt103_4_brasil(self,dset,dset2,dset3,label,precipitacao):
      initial_time, forecast_time, dateTimeObj = self.date_info(dset)
      initial_time = initial_time[0]

      plt.figure(figsize=(10,10))
      proj  =  ccrs.PlateCarree()
      ax = plt.axes(projection=proj)
      self.america_do_sul(ax, dset, proj)
      self.axis_AS(ax, dset)

      # self.sudeste(ax, dset, proj)
      # self.axis_sudeste(ax, dset)

      time = f'{label}T21:00:00.000000000'

      ##############
      # PRECIPITACAO
      ##############
      PRECIPITACAO = precipitacao # argumento da função
      dia_acumulado = label # dia acumulado

      #######
      # msl
      #######
      msl = dset3.msl.loc[time,:,:]/100

      ######################
      # ESPESSURA 1000 - 500
      ######################
      espessura = Calc().thickness(dset,time)

      ################################
      # Prepara u e v do vento
      ################################
      U = np.array(dset.u.loc[time,1000,::self.step,::self.step])*3.6
      V = np.array(dset.v.loc[time,1000,::self.step,::self.step])*3.6


      ###############################
      # PRECIPITAÇÃO ACUMULADA DE 24H
      ###############################
      colors = ('#00ff00','#1c8c1c','#006400','#121271','#3ee1d1','#00cfd2','#80ffd5','#ff6345','#b31c1c','#a62525','#ffa700','#ffd800')
      plt.contourf(grib.longitude, grib.latitude, PRECIPITACAO,
                    len(colors)-1,
                    colors=colors,
                    levels=[2,5,10,15,20,25,30,40,50,75,90,110],
                    transform=ccrs.PlateCarree(),
                    extend = 'max')

      cb = plt.colorbar(ax=ax, orientation="vertical", pad=0,ticks=[2,5,10,15,20,25,30,40,50,75,90,110], aspect=16, shrink=0.6)
      cb.set_label('precipitação acumulada (24h)', fontsize=self.size)
      cb.ax.tick_params(labelsize=self.size)

      # ####################################
      # # Pressão a Nível Médio do Mar
      # ####################################
      line = plt.contour(dset.longitude, dset.latitude,
                  msl,
                  colors ='gray',
                  transform=ccrs.PlateCarree(),
                  levels =  np.arange(np.min(msl),np.max(msl),3),
                  extend = None)

      idline = ax.clabel(line, inline=True, fontsize=10, fmt='%d')

      ####################################
      # espessura - linhas
      ####################################
      line = plt.contour(dset.longitude[::5], dset.latitude[::5],
                         espessura[::5,::5],
                         colors ='blue',
                         transform=ccrs.PlateCarree(),
                         levels = np.arange(516,595,6),
                         linestyles='--',
                         extend = None)

      idline = ax.clabel(line, inline=True, fontsize=10, fmt='%d')

      line = plt.contour(dset.longitude[::8], dset.latitude[::8],
                         espessura[::8,::8],
                         colors ='red',
                         transform=ccrs.PlateCarree(),
                         levels = [540],
                         extend = None)
      ####################################
      # Vento SFC - barbelas
      ####################################
      plt.barbs(dset.longitude[::self.step], dset.latitude[::self.step],
                U,V)

      #########
      # LEGENDA
      #########
      ax.text(-65, -66.5,'PNMM + PREC. ACUM. 24h + VENTO 10m Km/h',fontsize=10)

      #########
      # TITULO
      #########
      ax.set_title(f'GPT 103',
                   fontsize=self.size, loc='center')

      self.date_info(dset)
      # ax.set_title(f'{dia_acumulado}', fontsize=self.size, loc='right')

      self.set_initial_time(ax,dset,initial_time)

      time = f'{label}T21:00:00.000000000'
      date = datetime(int(time[0:4]), int(time[5:7]), int(time[8:10])) + timedelta(days=+1)

      ano = date.year
      mes = date.month
      dia = date.day

      time1 = f'{ano}-{mes:02}-{dia}T00:00:00.000000000'
      ax.set_title(f'VÁLIDO EM: {time1}'[:-13]+' UTC', fontsize=self.size, loc='right')

      self.save_figure(f'gpt103_4_brasil_{time}')
      plt.show()

In [ ]:
# Plot().gpt18_umidade(grib,grib1,f'{ano}-{mes}-{dia}T00:00:00.000000000')         # FUNCIONOU

# Plot().gpt103_1_brasil(grib,grib3,grib1,f'{ano}-{mes}-{dia}T00:00:00.000000000') # FUNCIONOU
# Plot().gpt103_2_brasil(grib,grib3,grib1,f'{ano}-{mes}-{dia}T00:00:00.000000000') # FUNCIONOU
# Plot().gpt103_3_brasil(grib,grib3,grib1,f'{ano}-{mes}-{dia}T00:00:00.000000000') # FUNCIONOU

# label, precipitacao = Calc().precAccum24(grib2)
# Plot().gpt103_4_brasil(grib,grib2,grib1,label[0],precipitacao[0])                # FUNCIONOU

# Plot().gpt221_brasil(grib,grib4,f'{ano}-{mes}-{dia}T00:00:00.000000000')         # FUNCIONOU (MUCAPE)
# Plot().gpt221_sudeste(grib,grib4,f'{ano}-{mes}-{dia}T00:00:00.000000000')        # FUNCIONOU (MUCAPE)


Plot().gpt45_sudeste(grib,grib3,grib1,f'{ano}-{mes}-{dia}T00:00:00.000000000')


<ipython-input-8-4267bb8a7832>:749: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  initial_time = initial_time[0]
/usr/local/lib/python3.11/dist-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/50m_physical/ne_50m_coastline.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)
/usr/local/lib/python3.11/dist-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/50m_cultural/ne_50m_admin_0_boundary_lines_land.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)
/usr/local/lib/python3.11/dist-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/10m_cultural/ne_10m_admin_1_states_provinces_lines.zip
  warnings.warn(f'Downloading: {url}',

In [ ]:
time = np.array(grib.time)
time = [str(x) for x in time]
##########
# GPT 18 #
##########
[Plot().gpt18_umidade(grib,grib1,x) for x in time]         # FUNCIONOU

# Plot().gpt18_umidade(grib,grib1,f'{ano}-{mes}-{dia}T00:00:00.000000000')         # FUNCIONOU

###########
# GPT 221 #
###########
# [Plot().gpt221_brasil(grib,grib2,x) for x in time]         # FUNCIONOU
# [Plot().gpt221_sudeste(grib,grib2,x) for x in time]        # FUNCIONOU

# Plot().gpt221_brasil(grib,grib2,f'{ano}-{mes}-{dia}T00:00:00.000000000')         # FUNCIONOU
# Plot().gpt221_sudeste(grib,grib2,f'{ano}-{mes}-{dia}T00:00:00.000000000')        # FUNCIONOU

###########
# GPT 103 #
###########
[Plot().gpt103_1_brasil(grib,grib3,grib1,x) for x in time] # FUNCIONOU
[Plot().gpt103_2_brasil(grib,grib3,grib1,x) for x in time] # FUNCIONOU
[Plot().gpt103_3_brasil(grib,grib3,grib1,x) for x in time] # FUNCIONOU - VERIFICAR SE AS CORES DO JATO ESTÃO CERTAS

label, precipitacao = Calc().precAccum24(grib2)
[Plot().gpt103_4_brasil(grib,grib2,grib1,label[x],precipitacao[x]) for x in range(len(label[:-2]))]  # FUNCIONOU - AUMENTAR O DOMÍNIO

# Plot().gpt103_1_brasil(grib,grib3,grib1,f'{ano}-{mes}-{dia}T00:00:00.000000000') # FUNCIONOU
# Plot().gpt103_2_brasil(grib,grib3,grib1,f'{ano}-{mes}-{dia}T00:00:00.000000000') # FUNCIONOU
# Plot().gpt103_3_brasil(grib,grib3,grib1,f'{ano}-{mes}-{dia}T00:00:00.000000000') # FUNCIONOU - VERIFICAR SE AS CORES DO JATO ESTÃO CERTAS
# Plot().gpt103_4_brasil(grib,grib2,grib1,label[0],precipitacao[0])                # FUNCIONOU

##########
# GPT 45 #
##########
[Plot().gpt45_sudeste(grib,grib3,grib1,x) for x in time]  # FUNCIONOU - TERMINAR O AZUL E DESCOBRIR O QUE É O ROXO

<ipython-input-8-4267bb8a7832>:155: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  initial_time = initial_time[0]
/usr/local/lib/python3.11/dist-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/110m_physical/ne_110m_coastline.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)
/usr/local/lib/python3.11/dist-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/110m_cultural/ne_110m_admin_0_boundary_lines_land.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)
<ipython-input-8-4267bb8a7832>:155: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To acces

In [ ]:
import shutil
import os

def zip_folder(folder_path, zip_path):
  """Zips a folder.

  Args:
    folder_path: Path to the folder to zip.
    zip_path: Path to the output zip file.
  """
  try:
    shutil.make_archive(zip_path, 'zip', folder_path)
    print(f"Folder '{folder_path}' zipped successfully to '{zip_path}.zip'")
  except FileNotFoundError:
    print(f"Error: Folder '{folder_path}' not found.")
  except Exception as e:
    print(f"An error occurred: {e}")

# Example usage: Assuming figures folder is in the current directory
figures_folder_path = "figures"  # Replace with your actual folder path
zip_file_path = f"figures_ecmwf_{ano}-{mes}-{dia} {hora_prev}z" # Output zip file name (without .zip extension)


if os.path.exists(figures_folder_path):
  zip_folder(figures_folder_path, zip_file_path)

  # Download the zip file (optional)
  from google.colab import files
  files.download(f"{zip_file_path}.zip")
else:
  print(f"Error: Folder '{figures_folder_path}' not found.")
